<a href="https://colab.research.google.com/github/pavankumarallu/K_ML/blob/main/Hand_Gesture_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts

import tensorflow as tf


from sklearn.metrics import classification_report, confusion_matrix

In [3]:
dfs = []

for label in ['0','1','2','3']:
  dfs.append(pd.read_csv('/content/'+label+'.csv'))


In [4]:
dfs[0]

,26.0,4.0,5.0,8.0,-1.0,-13.0,-109.0,-66.0,-9.0,2.0,4.0.1,13.0,-18.0,-30.0,-119.0,-45.0,17.0,6.0,-1.0.1,-9.0.1,27.0,20.0,91.0,71.0,-26.0,-1.0.2,0.0,1.0,13.0.1,20.0.1,-62.0,-19.0,29.0,2.0.1,2.0.2,0.0.1,-23.0,-1.0.3,-80.0,4.0.2,-7.0,-6.0,-12.0,-27.0,-11.0,-16.0,-67.0,-8.0,-27.0.1,1.0.1,1.0.2,13.0.2,-8.0.1,-11.0.1,21.0,-28.0,61.0,4.0.3,8.0.1,5.0.1,4.0.4,-7.0.1,-59.0,16.0,0
0,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,0.0,7.0,-31.0,-33.0,-48.0,-52.0,34.0,10.0,5.0,-9.0,23.0,63.0,76.0,49.0,-27.0,-7.0,-7.0,-3.0,0.0,4.0,-111.0,-2.0,-7.0,7.0,11.0,11.0,2.0,2.0,76.0,3.0,7.0,-2.0,0.0,-6.0,21.0,-3.0,0.0,-8.0,-40.0,-4.0,-1.0,6.0,-2.0,-2.0,-105.0,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
1,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,4.0,16.0,24.0,9.0,33.0,34.0,-1.0,-4.0,-3.0,-6.0,-7.0,-11.0,-85.0,-12.0,-10.0,-2.0,-3.0,-2.0,-8.0,-30.0,-16.0,-22.0,-21.0,-7.0,-8.0,-12.0,-14.0,-18.0,-76.0,-44.0,27.0,7.0,4.0,-4.0,-10.0,23.0,127.0,69.0,-25.0,-12.0,-9.0,1.0,18.0,-33.0,-128.0,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
2,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,-3.0,-4.0,18.0,21.0,-9.0,2.0,-8.0,0.0,3.0,5.0,-14.0,-15.0,13.0,-13.0,11.0,-4.0,-5.0,-8.0,-30.0,-27.0,-128.0,-22.0,-3.0,-5.0,-8.0,0.0,25.0,-9.0,-28.0,2.0,-22.0,8.0,8.0,-2.0,2.0,12.0,53.0,20.0,28.0,-9.0,-5.0,-5.0,-7.0,-7.0,-54.0,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
3,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,4.0,13.0,13.0,-16.0,-45.0,-7.0,-84.0,-8.0,-3.0,-4.0,-36.0,-26.0,-128.0,-55.0,75.0,3.0,-8.0,-12.0,14.0,12.0,87.0,56.0,-3.0,2.0,0.0,-7.0,4.0,15.0,-13.0,0.0,-15.0,1.0,3.0,8.0,-2.0,-6.0,-55.0,-24.0,1.0,5.0,5.0,2.0,11.0,12.0,60.0,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0
4,15.0,-5.0,-5.0,-15.0,12.0,-22.0,-38.0,36.0,9.0,6.0,-2.0,-4.0,6.0,61.0,84.0,70.0,23.0,4.0,2.0,8.0,4.0,14.0,-71.0,13.0,-46.0,-3.0,-6.0,-3.0,-8.0,-16.0,-99.0,-37.0,14.0,-1.0,0.0,2.0,-4.0,-16.0,13.0,-7.0,-23.0,-4.0,5.0,16.0,4.0,-7.0,21.0,-16.0,47.0,-2.0,-7.0,-28.0,13.0,-11.0,22.0,-26.0,5.0,6.0,6.0,11.0,5.0,30.0,-48.0,25.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2904,-12.0,0.0,0.0,-7.0,-4.0,-3.0,-1.0,-5.0,4.0,-1.0,-2.0,-1.0,-9.0,-10.0,-8.0,-11.0,2.0,1.0,1.0,1.0,11.0,-13.0,-89.0,-29.0,-5.0,-2.0,-2.0,0.0,-11.0,-5.0,91.0,59.0,-7.0,-1.0,0.0,-7.0,1.0,11.0,43.0,9.0,23.0,3.0,1.0,0.0,9.0,13.0,19.0,15.0,7.0,0.0,-2.0,-2.0,0.0,2.0,44.0,12.0,-3.0,0.0,2.0,-1.0,-1.0,4.0,-30.0,-20.0,0
2905,-9.0,-1.0,-2.0,1.0,-13.0,-14.0,-59.0,-4.0,-9.0,0.0,0.0,-1.0,12.0,3.0,-8.0,-9.0,7.0,0.0,1.0,-2.0,-16.0,-19.0,-48.0,-7.0,-17.0,-2.0,-2.0,-2.0,6.0,-5.0,10.0,1.0,7.0,3.0,3.0,5.0,13.0,17.0,37.0,0.0,-8.0,-4.0,-1.0,-1.0,-8.0,-16.0,3.0,-21.0,7.0,2.0,3.0,7.0,19.0,24.0,74.0,20.0,-4.0,0.0,0.0,0.0,-21.0,-10.0,-14.0,-29.0,0
2906,3.0,-2.0,-2.0,4.0,18.0,-5.0,-31.0,7.0,-3.0,-3.0,-4.0,-7.0,0.0,-4.0,-22.0,-5.0,-14.0,1.0,2.0,-6.0,-9.0,-9.0,-48.0,5.0,7.0,-9.0,-6.0,-5.0,-17.0,-21.0,-51.0,-58.0,11.0,3.0,3.0,0.0,0.0,6.0,80.0,15.0,-11.0,-3.0,0.0,-2.0,-9.0,19.0,-60.0,-5.0,7.0,-1.0,-8.0,-18.0,7.0,12.0,93.0,34.0,-7.0,4.0,4.0,2.0,1.0,4.0,-55.0,-4.0,0
2907,-2.0,-2.0,-2.0,1.0,-3.0,-17.0,-33.0,-13.0,3.0,2.0,2.0,6.0,0.0,-13.0,-7.0,5.0,-2.0,-4.0,1.0,6.0,-5.0,-2.0,-33.0,-5.0,-2.0,2.0,0.0,-3.0,17.0,12.0,56.0,0.0,-3.0,-2.0,-2.0,-8.0,-5.0,-23.0,-24.0,10.0,1.0,1.0,-1.0,3.0,6.0,16.0,46.0,11.0,3.0,-2.0,-2.0,-4.0,-12.0,-4.0,-11.0,-4.0,0.0,3.0,0.0,5.0,9.0,8.0,-13.0,11.0,0


In [5]:
for df in dfs:
  df.columns = list(range(len(df.columns)))

In [6]:
dfs[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
0,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,0.0,7.0,-31.0,-33.0,-48.0,-52.0,34.0,10.0,5.0,-9.0,23.0,63.0,76.0,49.0,-27.0,-7.0,-7.0,-3.0,0.0,4.0,-111.0,-2.0,-7.0,7.0,11.0,11.0,2.0,2.0,76.0,3.0,7.0,-2.0,0.0,-6.0,21.0,-3.0,0.0,-8.0,-40.0,-4.0,-1.0,6.0,-2.0,-2.0,-105.0,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
1,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,4.0,16.0,24.0,9.0,33.0,34.0,-1.0,-4.0,-3.0,-6.0,-7.0,-11.0,-85.0,-12.0,-10.0,-2.0,-3.0,-2.0,-8.0,-30.0,-16.0,-22.0,-21.0,-7.0,-8.0,-12.0,-14.0,-18.0,-76.0,-44.0,27.0,7.0,4.0,-4.0,-10.0,23.0,127.0,69.0,-25.0,-12.0,-9.0,1.0,18.0,-33.0,-128.0,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
2,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,-3.0,-4.0,18.0,21.0,-9.0,2.0,-8.0,0.0,3.0,5.0,-14.0,-15.0,13.0,-13.0,11.0,-4.0,-5.0,-8.0,-30.0,-27.0,-128.0,-22.0,-3.0,-5.0,-8.0,0.0,25.0,-9.0,-28.0,2.0,-22.0,8.0,8.0,-2.0,2.0,12.0,53.0,20.0,28.0,-9.0,-5.0,-5.0,-7.0,-7.0,-54.0,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
3,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,4.0,13.0,13.0,-16.0,-45.0,-7.0,-84.0,-8.0,-3.0,-4.0,-36.0,-26.0,-128.0,-55.0,75.0,3.0,-8.0,-12.0,14.0,12.0,87.0,56.0,-3.0,2.0,0.0,-7.0,4.0,15.0,-13.0,0.0,-15.0,1.0,3.0,8.0,-2.0,-6.0,-55.0,-24.0,1.0,5.0,5.0,2.0,11.0,12.0,60.0,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0
4,15.0,-5.0,-5.0,-15.0,12.0,-22.0,-38.0,36.0,9.0,6.0,-2.0,-4.0,6.0,61.0,84.0,70.0,23.0,4.0,2.0,8.0,4.0,14.0,-71.0,13.0,-46.0,-3.0,-6.0,-3.0,-8.0,-16.0,-99.0,-37.0,14.0,-1.0,0.0,2.0,-4.0,-16.0,13.0,-7.0,-23.0,-4.0,5.0,16.0,4.0,-7.0,21.0,-16.0,47.0,-2.0,-7.0,-28.0,13.0,-11.0,22.0,-26.0,5.0,6.0,6.0,11.0,5.0,30.0,-48.0,25.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2904,-12.0,0.0,0.0,-7.0,-4.0,-3.0,-1.0,-5.0,4.0,-1.0,-2.0,-1.0,-9.0,-10.0,-8.0,-11.0,2.0,1.0,1.0,1.0,11.0,-13.0,-89.0,-29.0,-5.0,-2.0,-2.0,0.0,-11.0,-5.0,91.0,59.0,-7.0,-1.0,0.0,-7.0,1.0,11.0,43.0,9.0,23.0,3.0,1.0,0.0,9.0,13.0,19.0,15.0,7.0,0.0,-2.0,-2.0,0.0,2.0,44.0,12.0,-3.0,0.0,2.0,-1.0,-1.0,4.0,-30.0,-20.0,0
2905,-9.0,-1.0,-2.0,1.0,-13.0,-14.0,-59.0,-4.0,-9.0,0.0,0.0,-1.0,12.0,3.0,-8.0,-9.0,7.0,0.0,1.0,-2.0,-16.0,-19.0,-48.0,-7.0,-17.0,-2.0,-2.0,-2.0,6.0,-5.0,10.0,1.0,7.0,3.0,3.0,5.0,13.0,17.0,37.0,0.0,-8.0,-4.0,-1.0,-1.0,-8.0,-16.0,3.0,-21.0,7.0,2.0,3.0,7.0,19.0,24.0,74.0,20.0,-4.0,0.0,0.0,0.0,-21.0,-10.0,-14.0,-29.0,0
2906,3.0,-2.0,-2.0,4.0,18.0,-5.0,-31.0,7.0,-3.0,-3.0,-4.0,-7.0,0.0,-4.0,-22.0,-5.0,-14.0,1.0,2.0,-6.0,-9.0,-9.0,-48.0,5.0,7.0,-9.0,-6.0,-5.0,-17.0,-21.0,-51.0,-58.0,11.0,3.0,3.0,0.0,0.0,6.0,80.0,15.0,-11.0,-3.0,0.0,-2.0,-9.0,19.0,-60.0,-5.0,7.0,-1.0,-8.0,-18.0,7.0,12.0,93.0,34.0,-7.0,4.0,4.0,2.0,1.0,4.0,-55.0,-4.0,0
2907,-2.0,-2.0,-2.0,1.0,-3.0,-17.0,-33.0,-13.0,3.0,2.0,2.0,6.0,0.0,-13.0,-7.0,5.0,-2.0,-4.0,1.0,6.0,-5.0,-2.0,-33.0,-5.0,-2.0,2.0,0.0,-3.0,17.0,12.0,56.0,0.0,-3.0,-2.0,-2.0,-8.0,-5.0,-23.0,-24.0,10.0,1.0,1.0,-1.0,3.0,6.0,16.0,46.0,11.0,3.0,-2.0,-2.0,-4.0,-12.0,-4.0,-11.0,-4.0,0.0,3.0,0.0,5.0,9.0,8.0,-13.0,11.0,0


In [7]:
data = pd.concat([df for df in dfs],axis = 0).reset_index(drop = True)

In [8]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
0,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,0.0,7.0,-31.0,-33.0,-48.0,-52.0,34.0,10.0,5.0,-9.0,23.0,63.0,76.0,49.0,-27.0,-7.0,-7.0,-3.0,0.0,4.0,-111.0,-2.0,-7.0,7.0,11.0,11.0,2.0,2.0,76.0,3.0,7.0,-2.0,0.0,-6.0,21.0,-3.0,0.0,-8.0,-40.0,-4.0,-1.0,6.0,-2.0,-2.0,-105.0,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
1,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,4.0,16.0,24.0,9.0,33.0,34.0,-1.0,-4.0,-3.0,-6.0,-7.0,-11.0,-85.0,-12.0,-10.0,-2.0,-3.0,-2.0,-8.0,-30.0,-16.0,-22.0,-21.0,-7.0,-8.0,-12.0,-14.0,-18.0,-76.0,-44.0,27.0,7.0,4.0,-4.0,-10.0,23.0,127.0,69.0,-25.0,-12.0,-9.0,1.0,18.0,-33.0,-128.0,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
2,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,-3.0,-4.0,18.0,21.0,-9.0,2.0,-8.0,0.0,3.0,5.0,-14.0,-15.0,13.0,-13.0,11.0,-4.0,-5.0,-8.0,-30.0,-27.0,-128.0,-22.0,-3.0,-5.0,-8.0,0.0,25.0,-9.0,-28.0,2.0,-22.0,8.0,8.0,-2.0,2.0,12.0,53.0,20.0,28.0,-9.0,-5.0,-5.0,-7.0,-7.0,-54.0,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
3,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,4.0,13.0,13.0,-16.0,-45.0,-7.0,-84.0,-8.0,-3.0,-4.0,-36.0,-26.0,-128.0,-55.0,75.0,3.0,-8.0,-12.0,14.0,12.0,87.0,56.0,-3.0,2.0,0.0,-7.0,4.0,15.0,-13.0,0.0,-15.0,1.0,3.0,8.0,-2.0,-6.0,-55.0,-24.0,1.0,5.0,5.0,2.0,11.0,12.0,60.0,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0
4,15.0,-5.0,-5.0,-15.0,12.0,-22.0,-38.0,36.0,9.0,6.0,-2.0,-4.0,6.0,61.0,84.0,70.0,23.0,4.0,2.0,8.0,4.0,14.0,-71.0,13.0,-46.0,-3.0,-6.0,-3.0,-8.0,-16.0,-99.0,-37.0,14.0,-1.0,0.0,2.0,-4.0,-16.0,13.0,-7.0,-23.0,-4.0,5.0,16.0,4.0,-7.0,21.0,-16.0,47.0,-2.0,-7.0,-28.0,13.0,-11.0,22.0,-26.0,5.0,6.0,6.0,11.0,5.0,30.0,-48.0,25.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11669,-3.0,-1.0,-1.0,-1.0,-28.0,20.0,5.0,0.0,-5.0,0.0,2.0,5.0,-10.0,-17.0,-3.0,-2.0,12.0,2.0,2.0,2.0,-4.0,-28.0,1.0,-6.0,-5.0,-1.0,1.0,0.0,11.0,9.0,-4.0,14.0,2.0,-2.0,-2.0,-8.0,-17.0,1.0,-2.0,6.0,-14.0,-3.0,-4.0,-2.0,27.0,18.0,-6.0,-17.0,-4.0,1.0,-2.0,1.0,13.0,-12.0,-3.0,-3.0,1.0,4.0,3.0,4.0,-51.0,-49.0,5.0,-9.0,3
11670,-13.0,-5.0,-4.0,-3.0,-4.0,-24.0,-10.0,-8.0,20.0,9.0,9.0,7.0,5.0,50.0,31.0,18.0,-3.0,-3.0,1.0,-2.0,17.0,-6.0,-10.0,-18.0,-11.0,-5.0,-2.0,-2.0,10.0,21.0,-2.0,15.0,5.0,-2.0,-2.0,-6.0,-6.0,-50.0,-16.0,7.0,-12.0,-2.0,-3.0,1.0,21.0,48.0,12.0,-12.0,-1.0,3.0,2.0,5.0,-12.0,-8.0,5.0,6.0,-3.0,-3.0,-3.0,-5.0,-4.0,-45.0,-12.0,-15.0,3
11671,-1.0,-3.0,-1.0,1.0,30.0,38.0,-1.0,36.0,-10.0,1.0,4.0,0.0,-3.0,33.0,-2.0,-1.0,-1.0,2.0,-3.0,0.0,-38.0,-25.0,0.0,3.0,2.0,-1.0,0.0,-3.0,14.0,0.0,-1.0,-3.0,-3.0,-4.0,-3.0,-6.0,-2.0,13.0,-5.0,-19.0,-6.0,-5.0,-4.0,1.0,41.0,-10.0,-4.0,-9.0,3.0,3.0,2.0,3.0,1.0,50.0,12.0,14.0,-8.0,-4.0,-4.0,-4.0,-21.0,-29.0,-5.0,0.0,3
11672,1.0,4.0,4.0,5.0,9.0,-10.0,4.0,1.0,-2.0,-1.0,-1.0,-1.0,-5.0,12.0,1.0,-19.0,0.0,-4.0,-4.0,0.0,25.0,0.0,3.0,-5.0,1.0,1.0,-2.0,-4.0,-31.0,-17.0,-7.0,5.0,3.0,-5.0,-3.0,-1.0,4.0,-48.0,-13.0,-5.0,-14.0,6.0,5.0,6.0,-3.0,79.0,15.0,-6.0,-3.0,-1.0,-3.0,-1.0,-1.0,27.0,-2.0,-16.0,-3.0,0.0,-3.0,-5.0,-36.0,-90.0,3.0,5.0,3


In [9]:
data.isnull().sum().sum()

0

# **Splitting/Scaling**

In [10]:
X = data.drop(64,axis = 1).copy()
y = data[64].copy()

In [11]:
scaler = StandardScaler()

In [12]:
X = scaler.fit_transform(X)

In [13]:
x_train,x_test,y_train,y_test = tts(X,y,train_size = 0.7,random_state = 100)

In [14]:
x_train.shape , x_test.shape

((8171, 64), (3503, 64))

In [15]:
inputs = tf.keras.Input(shape=(X.shape[1],))
x = tf.keras.layers.Dense(1024,activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(1024,activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = tf.keras.layers.Dense(4,activation='softmax')(x)

model = tf.keras.Model(inputs,outputs)



In [16]:

model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_entropy',
    metrics = ['accuracy']
)

batch_size = 32
epochs = 50


In [ ]:
history = model.fit(x_train,y_train,validation_split=0.2,batch_size=32,epochs=25,callbacks=[tf.keras.callbacks.ReduceLROnPlateau()])

In [ ]:
plt.figure(figsize=(16,10))

plt.plot(range(25),history.history['loss'],label = 'Training Loss')
plt.plot(range(25),history.history['val_loss'],label = 'Validation Loss')

In [ ]:
y_true = np.array(y_test)
y_pred = np.array(list(map(lambda x: np.argmax(x),model.predict(x_test))))


In [ ]:
cm = confusion_matrix(y_true,y_pred)

plt.figure(figsize=(6,6))
sns.heatmap(cm,annot = True,fmt = 'g',cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
print(classification_report(y_true,y_pred))